In [ ]:
# 최초의 cnn 당시엔 relu가 없었기 때문에 sigmoid 사용하고, 에포크는 좀 줄여라 80퍼 이상은 나올것이다
import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets as ds

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 데이터 준비
(x_train, y_train), (x_test, y_test) = ds.mnist.load_data()
x_train=x_train.reshape(60000,28,28,1) # 2차원 구조로 변환 / 데이터 수
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

cnn=Sequential() # 순차구조 사용
cnn.add(Conv2D(6,(5,5),padding='same',activation='sigmoid',input_shape=(28,28,1)))# @ 5*5커널 6개 사용
cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
cnn.add(Conv2D(16,(5,5),padding='valid',activation='sigmoid'))
cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
cnn.add(Conv2D(120,(5,5),padding='valid',activation='sigmoid'))
cnn.add(Flatten()) # 1차원 구조로 변환
cnn.add(Dense(units=84,activation='sigmoid'))
cnn.add(Dense(units=10,activation='softmax')) # 여기까지 신경망 구조 설계

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy']) # 학습
cnn.fit(x_train,y_train,batch_size=128,epochs=15,validation_data=(x_test,y_test),verbose=2)

res=cnn.evaluate(x_test,y_test,verbose=0) # 예측
print('정확률=',res[1]*100)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
469/469 - 33s - 70ms/step - accuracy: 0.4942 - loss: 1.4969 - val_accuracy: 0.8835 - val_loss: 0.4455
Epoch 2/15
469/469 - 31s - 66ms/step - accuracy: 0.9170 - loss: 0.2968 - val_accuracy: 0.9430 - val_loss: 0.1877
Epoch 3/15
469/469 - 41s - 87ms/step - accuracy: 0.9537 - loss: 0.1576 - val_accuracy: 0.9667 - val_loss: 0.1134
Epoch 4/15
469/469 - 41s - 87ms/step - accuracy: 0.9679 - loss: 0.1081 - val_accuracy: 0.9744 - val_loss: 0.0843
Epoch 5/15
469/469 - 42s - 90ms/step - accuracy: 0.9748 - loss: 0.0851 - val_accuracy: 0.9799 - val_loss: 0.0701
Epoch 6/15
469/469 - 32s - 68ms/step - accuracy: 0.9786 - loss: 0.0713 - val_accuracy: 0.9818 - val_loss: 0.0615
Epoch 7/15
469/469 - 40s - 85ms/step - accuracy: 0.9817 - loss: 0.0629 - val_accuracy: 0.9824 - val_loss: 0.0548
Epoch 8/15
469/469 - 41s - 87ms/step - accuracy: 0.9842 - loss: 0.0542 - val_accuracy: 0.9827 - val_loss: 0.0536
Epoch 9/15
469/469 - 40s - 85ms/step - accuracy: 0.9857 - loss: 0.0486 - val_accuracy: 0.9845 - 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# CIFAR-10 데이터 로드
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 데이터 정규화 및 원-핫 인코딩
x_train, x_test = x_train.astype(np.float32) / 255.0, x_test.astype(np.float32) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 알렉스넷 모델 (CIFAR-10 맞춤)
model = Sequential()
model.add(Conv2D(24, (11,11), strides=1, padding='same', activation='sigmoid', input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(256, (5,5), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(384, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(384, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())
model.add(Dense(4096, activation='sigmoid'))  # 4096 -> 1024로 축소
model.add(Dropout(0.5))
model.add(Dense(4096, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))  # CIFAR-10 클래스 수

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# 체크포인트 저장 및 얼리 스토핑 설정
checkpoint = ModelCheckpoint('alexnet_cifar10.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_test, y_test),
                    callbacks=[checkpoint, early_stopping], verbose=2)

# 모델 평가
res = model.evaluate(x_test, y_test, verbose=0)
print('테스트 정확률 =', res[1] * 100)

# 학습 과정 시각화
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 10), output.shape=(None, 1000)

In [ ]:
# VGG 모델
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# CIFAR-10 데이터 로드
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 데이터 정규화 및 원-핫 인코딩
x_train, x_test = x_train.astype(np.float32) / 255.0, x_test.astype(np.float32) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# VGG 모델
model = Sequential()
model.add(Conv2D(64, (3,3), strides=1, padding='same', activation='sigmoid', input_shape=(32,32,3)))
model.add(Conv2D(64, (3,3), strides=1, padding='same', activation='sigmoid', input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(128, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(128, (3,3), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())
model.add(Dense(4096, activation='sigmoid'))  # 4096 -> 1024로 축소
model.add(Dropout(0.5))
model.add(Dense(4096, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))  # CIFAR-10 클래스 수 최종 10개의 클래스로 구별하기 때문에 최종 노드가 10이어야한다.

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# 체크포인트 저장 및 얼리 스토핑 설정
checkpoint = ModelCheckpoint('vgg_cifar10.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_test, y_test),
                    callbacks=[checkpoint, early_stopping], verbose=2)

# 모델 평가
res = model.evaluate(x_test, y_test, verbose=0)
print('테스트 정확률 =', res[1] * 100)

# 학습 과정 시각화
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Epoch 1/15


In [ ]:
# resnet의 핵심은 잔차함수! 미분을 계속해서 정전파를 이어가다보면 "기울기 소실"
# 문제가 발생하는데 ex0.005 -> 0 이런식으로 이를 해겷하기 위해 resnet이 사용되었다
# from tesorflow.keras.applications import ResNet50 // applications 안에 인셉션, 시퀸스, resnet 다 있다
# 구글넷을 직접 작성해보기.... 줴엔장...
